In [ ]:
import synapseclient

import pandas as pd
import great_expectations as gx

from agoradatatools.gx import GreatExpectationsRunner

context = gx.get_context(project_root_dir='../src/agoradatatools/great_expectations')

# Create Expectation Suite for Network Data

## Get Example Data File

In [ ]:
syn = synapseclient.Synapse()
syn.login()


In [ ]:
network_file = syn.get("syn17015361").path

## Create Validator Object on Data File

In [ ]:
df = pd.read_json(network_file)
nested_columns = []
df = GreatExpectationsRunner.convert_nested_columns_to_json(df, nested_columns)
validator = context.sources.pandas_default.read_dataframe(df)
validator.expectation_suite_name = "network"

## Add Expectations to Validator Object For Each Column

In [ ]:
# geneA_ensembl_gene_id
validator.expect_column_values_to_be_of_type("geneA_ensembl_gene_id", "str")
validator.expect_column_values_to_not_be_null("geneA_ensembl_gene_id")
validator.expect_column_values_to_match_regex("geneA_ensembl_gene_id", "^ENSG\d{11}$")

In [ ]:
# geneB_ensembl_gene_id
validator.expect_column_values_to_be_of_type("geneB_ensembl_gene_id", "str")
validator.expect_column_values_to_not_be_null("geneB_ensembl_gene_id")
validator.expect_column_values_to_match_regex("geneB_ensembl_gene_id", "^ENSG\d{11}$")

In [ ]:
# geneA_external_gene_name
validator.expect_column_values_to_be_of_type("geneA_external_gene_name", "str")
validator.expect_column_values_to_not_be_null("geneA_external_gene_name", mostly=0.99)
validator.expect_column_value_lengths_to_be_between("geneA_external_gene_name", 2, 100)

In [ ]:
# geneB_external_gene_name
validator.expect_column_values_to_be_of_type("geneB_external_gene_name", "str")
validator.expect_column_values_to_not_be_null("geneB_external_gene_name", mostly=0.99)
validator.expect_column_value_lengths_to_be_between("geneB_external_gene_name", 2, 100)

In [ ]:
# brainRegion
validator.expect_column_values_to_be_of_type("brainRegion", "str")
validator.expect_column_values_to_not_be_null("brainRegion")
validator.expect_column_values_to_be_in_set("brainRegion", ["CBE", "DLPFC", "FP", "IFG", "PHG", "STG", "TCX"])

In [ ]:
# multi-field
validator.expect_compound_columns_to_be_unique(["brainRegion", "geneA_ensembl_gene_id", "geneB_ensembl_gene_id"])

## Save Expectation Suite

In [ ]:
validator.save_expectation_suite(discard_failed_expectations=False)


## Create Checkpoint and View Results

In [ ]:
checkpoint = context.add_or_update_checkpoint(
    name="agora-test-checkpoint",
    validator=validator,
)
checkpoint_result = checkpoint.run()
context.view_validation_result(checkpoint_result)


## Build Data Docs - Click on Expectation Suite to View All Expectations

In [ ]:
context.build_data_docs()
context.open_data_docs()
